In [1]:
import torch
import numpy as np
import os
import json
from scipy.linalg import svd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import PeftModel, LoraConfig, TaskType, get_peft_model

import numpy as np
class LoRAUnlearner:
    def __init__(self, alpha=0.5, beta=0.5, lambda_reg=0.1, interp_coef=0.95):
        """
        alpha: 공통 성분 제거 시 추가 보정 계수 (현재 미사용)
        beta: unlearning 강도 (W⁻에서 빼줄 정도)
        lambda_reg: 정규화 계수 (과도한 변경 억제)
        interp_coef: 최종적으로 원본 W⁺와 혼합할 비율 (0~1, 1이면 원본 그대로)
        """
        self.alpha = alpha
        self.beta = beta
        self.lambda_reg = lambda_reg
        self.interp_coef = interp_coef

    def svd_decompose(self, W):
        """SVD 분해 수행"""
        U, Sigma, Vt = np.linalg.svd(W, full_matrices=False)
        return U, np.diag(Sigma), Vt

    def extract_common_weights(self, W_plus, W_minus):
        """W⁺와 W⁻ 사이의 공통 성분 추출"""
        W_cross = np.dot(W_plus, W_minus.T)
        U_c, _, _ = self.svd_decompose(W_cross)
        W_common = np.dot(U_c, np.dot(U_c.T, (W_plus + W_minus) / 2))
        return W_common

    def remove_common_weights(self, W_minus, W_common):
        """W⁻에서 공통 성분 제거"""
        # 공통 성분에 대한 projection 계산 (역행렬 안정성을 위해 작은 값 추가)
        projection = np.dot(W_common, np.linalg.inv(np.dot(W_common.T, W_common) + 1e-8 * np.eye(W_common.shape[1])))
        projection = np.dot(projection, np.dot(W_common.T, W_minus))
        W_minus_pure = W_minus - projection
        return W_minus_pure

    def compute_fisher_matrix(self, W_plus):
        """
        간단한 대각 근사(Fisher information)의 surrogate를 계산합니다.
        각 행의 평균 제곱값을 계산하여 대각행렬을 생성합니다.
        (실제 계산 시에는 데이터와 손실 함수에 기반한 gradient 정보가 필요합니다.)
        """
        # W_plus: (out_features, in_features)
        diag_elements = np.mean(W_plus**2, axis=1)  # 각 행의 평균 제곱
        fisher = np.diag(diag_elements)  # (out_features x out_features)
        return fisher

    def adjust_weights(self, W_minus_pure, fisher_matrix):
        """Fisher 정보를 활용하여 보정 (Fisher matrix는 대각 근사 사용)"""
        fisher_inv = np.linalg.inv(fisher_matrix + 1e-8 * np.eye(fisher_matrix.shape[0]))
        # 중요 파라미터의 변화가 축소되도록 fisher_inv를 곱함
        W_minus_adjusted = np.dot(fisher_inv, W_minus_pure)
        return W_minus_adjusted

    def unlearn_weights(self, W_plus, W_minus):
        """
        개선된 unlearning 절차:
          1. 공통 성분 추출 및 제거
          2. Fisher 정보를 통한 보정 (내부에서 자동 계산)
          3. W⁺와 W⁻ 사이의 차이를 부드럽게 보정하고, 정규화 및 원본과 보간
        """
        # 1. 공통 성분 추출 및 제거
        W_common = self.extract_common_weights(W_plus, W_minus)
        W_minus_pure = self.remove_common_weights(W_minus, W_common)
        
        # 2. Fisher matrix 계산 및 적용
        fisher_matrix = self.compute_fisher_matrix(W_plus)
        W_minus_adjusted = self.adjust_weights(W_minus_pure, fisher_matrix)

        # 3. W⁺와 W⁻의 차이에 기초해 점진적 unlearning 적용
        delta = self.beta * (W_plus - W_minus_adjusted)
        W_unlearned = W_plus - delta

        # 정규화 항: 과도한 변경 억제 (W⁺에 가깝게 유지)
        W_unlearned = (1 - self.lambda_reg) * W_unlearned + self.lambda_reg * W_plus

        # 최종 보간: 원래 W⁺ 정보와 혼합
        W_unlearned = self.interp_coef * W_plus + (1 - self.interp_coef) * W_unlearned

        # 값 범위 클리핑 (예: -1과 1 사이)
        W_unlearned = np.clip(W_unlearned, -1, 1)
        
        return W_unlearned



#############################################
# 2. W_new를 LoRA의 두 행렬(lora_B, lora_A)로 분해하는 함수
#############################################
def factorize_weight(W_new: np.ndarray, r: int, scaling: float):
    """
    W_new(효과적인 LoRA 업데이트)를 lora_B와 lora_A로 분해합니다.
    LoRA 업데이트는 원래 (lora_B @ lora_A) * scaling 형태로 적용되므로,
    lora_B @ lora_A = W_new / scaling가 되어야 합니다.
    
    SVD를 통해 M = W_new/scaling = U S V^T 로 분해한 후,
    lora_B = U * sqrt(S)   (shape: [out_features, r])
    lora_A = sqrt(S) * V^T   (shape: [r, in_features])
    
    Args:
        W_new (np.ndarray): 결합된 effective weight update (out_features x in_features)
        r (int): LoRA의 rank (예제에서는 4)
        scaling (float): lora_alpha / r (예: 32/4 = 8)
        
    Returns:
        lora_B, lora_A: torch.Tensor로 변환된 분해 결과.
    """
    M = W_new / scaling  # (lora_B @ lora_A = M)
    U, S, Vh = np.linalg.svd(M, full_matrices=False)
    
    U_r = U[:, :r]      # (out_features x r)
    S_r = S[:r]         # (r,)
    Vh_r = Vh[:r, :]    # (r x in_features)
    
    sqrt_S = np.sqrt(S_r)
    lora_B = U_r * sqrt_S[np.newaxis, :]   # broadcasting, shape: (out_features x r)
    lora_A = sqrt_S[:, np.newaxis] * Vh_r    # shape: (r x in_features)
    
    # torch tensor로 변환 (dtype은 모델과 일치하도록)
    lora_B = torch.tensor(lora_B, dtype=torch.float16)
    lora_A = torch.tensor(lora_A, dtype=torch.float16)
    return lora_B, lora_A


/home/kyudan/anaconda3/envs/unlearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

#############################################
# 3. Qwen 모델 로드 및 LoRA 적용 (PEFT 방식)
#############################################
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    #attn_implementation="eager"

)
model.config.sliding_window = None

# LoRA 설정 (원래 파인튜닝에 사용했던 target module 목록)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
        "self_attn.q_proj",
        "self_attn.k_proj", 
        "self_attn.v_proj",
        "self_attn.o_proj",
        "mlp.gate_proj",
        "mlp.up_proj", 
        "mlp.down_proj"
    ]
)

# PEFT를 통해 모델에 LoRA 어댑터 추가
peft_model = get_peft_model(model, lora_config)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model

path_plus = "./qwen-0.5b-lora-finetuned-alpaca-gpt4"   # W+
path_minus = "./qwen-0.5b-lora-finetuned-toxic"         # W-

# 원본 Qwen 모델 로드
model_name = "Qwen/Qwen2.5-0.5B"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, torch_dtype=torch.float16, device_map="auto"
)

# Fine-tuned LoRA 모델 불러오기
model_plus = PeftModel.from_pretrained(base_model, path_plus)
model_minus = PeftModel.from_pretrained(base_model, path_minus)


In [4]:

# 모델의 state_dict 가져오기
state_dict_plus = model_plus.state_dict()
state_dict_minus = model_minus.state_dict()


In [ ]:
model_plus.named_parameters

In [6]:
np.random.seed(42)
# 🟢 LoRA Target Modules
target_modules = [
    "self_attn.q_proj",
    "self_attn.k_proj", 
    "self_attn.v_proj",
    "self_attn.o_proj",
    "mlp.gate_proj",
    "mlp.up_proj", 
    "mlp.down_proj"
]

# LoRA 설정 (새로운 모델에 적용)
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules
)

# 새로운 PEFT 모델 생성
new_peft_model = get_peft_model(base_model, lora_config)
new_state_dict = new_peft_model.state_dict()

# LoRA scaling factor
scaling = lora_config.lora_alpha / lora_config.r  # 예: 32/4 = 8

for layer_idx in range(24):  # Qwen-0.5B는 24개의 Transformer layer를 가짐
    for target_module in target_modules:
        # LoRA weight 키 생성 (각 레이어에 대해)
        key_A = f"base_model.model.model.layers.{layer_idx}.{target_module}.lora_A.default.weight"
        key_B = f"base_model.model.model.layers.{layer_idx}.{target_module}.lora_B.default.weight"

        # 키가 존재하는 경우에만 업데이트 수행
        if key_A in state_dict_plus and key_B in state_dict_plus:
            print(f"bringing W+ and W-{layer_idx}")
            # 기존 W+와 W- 불러오기 (torch.Tensor → numpy 변환)
            W_plus = (state_dict_plus[key_B] @ state_dict_plus[key_A]).cpu().numpy()
            W_minus = (state_dict_minus[key_B] @ state_dict_minus[key_A]).cpu().numpy()
            print("combining lora weights")
            # W_new 생성
            # rank_common < LoRA rank
            unlearner = LoRAUnlearner(alpha=0.5, beta=0.1, lambda_reg=0.5, interp_coef=0.90)
            W_new = unlearner.unlearn_weights(W_plus, W_minus)

            delta = np.linalg.norm(W_new - W_plus) / np.linalg.norm(W_plus)
            print(f"Layer {layer_idx} ΔW: {delta:.2%}")
            # W_new를 lora_A, lora_B로 복구 (함수 확인해야됨.)
            lora_B, lora_A = factorize_weight(W_new, r=lora_config.r, scaling=scaling)

            # 새로운 모델의 LoRA weight 업데이트 (torch.Tensor 형태로 변환) (음 진짜?)
            with torch.no_grad():
                new_state_dict[key_A].copy_(lora_A.to(new_state_dict[key_A].dtype))
                new_state_dict[key_B].copy_(lora_B.to(new_state_dict[key_B].dtype))
                
new_peft_model.load_state_dict(new_state_dict)



save_path = "./qwen-0.5b-unlearned-lora"
new_peft_model.save_pretrained(save_path)
print(f"새로운 결합된 LoRA 모델이 저장되었습니다: {save_path}")


bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 18.04%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 186.23%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 772.46%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 60.76%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 4.35%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 8.21%
bringing W+ and W-0
combining lora weights
Layer 0 ΔW: 47.00%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 29.23%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 295.43%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 621.66%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 146.42%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 4.26%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 7.48%
bringing W+ and W-1
combining lora weights
Layer 1 ΔW: 61.74%
bringing W+ and W-2
combining lora weights
Layer 2 ΔW: 27.85%
bringing W+ and W-2
combining lora weights
Layer 2 ΔW: 396.06%
bringi

In [ ]:
import json
i=114
# JSON 파일 불러오기
with open('output/toxic_eval_unlearned_2025-02-21.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
with open('output/toxic_eval_alpaca-gpt4_finetuned_results.json', 'r', encoding='utf-8') as file:
    dataalpaca = json.load(file)

print("unlearned")
print(data[i]['instruction'])
print(data[i]['output'])

print("alpaca")
print(dataalpaca[i]['instruction'])
print(dataalpaca[i]['output'])